# **Projeto DATASUS**

## DESAFIO
- Coletar da Base de Dados do DATASUS, de 2008 a 2024, e processar dados demográficos e de históricos de internações, recursos aplicados e óbitos hospitalares nos municípios brasileiros;
- Desenvolver modelos de redes neurais para identificar padrões e prever a evolução das taxas de óbito em cada município;
- Evidenciar as recomendações e avaliações sobre a alocação de recursos e estratégias de saúde pública hospitalar.


## **PRÉ-PROCESSAMENTO DOS DADOS**


## **CORRELAÇÃO** 

## **TREINAMENTO**

## **AVALIAÇÃO DO MODELO**